In [ ]:
!pip install google-genai sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 51.8 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from google import genai
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer , CrossEncoder
from IPython.display import Markdown
import faiss
#userdata.get('anime')

In [ ]:
api = userdata.get('anime')
model = 'gemini-2.5-flash'
path = '/content/anime_enquiry_dataset.csv'

In [ ]:
client = genai.Client(api_key=api)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
df = pd.read_csv(path)
df.head()

,question,answer
0,what is anime?,anime is japanese animated content
1,where did anime originate?,anime originated in japan
2,what is the meaning of shonen anime?,shonen anime targets young male audiences
3,what is shoujo anime?,shoujo anime targets young female audiences
4,what is seinen anime?,seinen anime targets adult male audiences


In [ ]:
document = [f"Q: {q}\nA: {a}" for  q, a in zip(df['question'], df['answer'])]

In [ ]:
embeddings = embedding_model.encode(document)
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings.astype('float'))
display(embeddings)

array([[-0.01726871,  0.00670018, -0.01891298, ...,  0.0437298 ,
         0.03086236,  0.01458868],
       [-0.04665253,  0.04323328, -0.00053733, ...,  0.050395  ,
         0.05089951,  0.02827998],
       [ 0.01266319,  0.01140079,  0.06460108, ...,  0.01586192,
        -0.04466379,  0.01556205],
       ...,
       [-0.05780292, -0.00813608,  0.03443282, ...,  0.04385267,
        -0.07042401,  0.01490764],
       [ 0.00075353,  0.04610914,  0.00033883, ...,  0.07448705,
        -0.00263381,  0.03275053],
       [ 0.01349615,  0.05968598,  0.02254636, ...,  0.04786095,
        -0.02900074,  0.02403436]], dtype=float32)

In [ ]:
def best_doc_find(query):
  query_vec = embedding_model.encode([query])
  faiss.normalize_L2(query_vec)
  distance, indices = index.search(
      np.array(query_vec).astype('float32'), k=5)
  candidates = [document[i] for i in indices[0]]

  pairs = [[query, doc] for doc in candidates]
  scores = rerank_model.predict(pairs)
  best_doc = candidates[np.argmax(scores)]
  return best_doc

In [ ]:
def best_doc_find(query):
  query_vec = embedding_model.encode([query])
  faiss.normalize_L2(query_vec)
  distance, indices = index.search(
      np.array(query_vec).astype('float32'), k=5)
  candidates = [document[i] for i in indices[0]]

  pairs = [[query, doc] for doc in candidates]
  scores = rerank_model.predict(pairs)
  best_doc = candidates[np.argmax(scores)]
  return best_doc

In [ ]:
chat_history = []
def get_rag_response(query):
  global chat_history
  best_doc = best_doc_find(query)
  memory_context = "\n".join(chat_history[-4:])
  prompt = f"""
 Histry: {memory_context}
 Context: {best_doc}
 User Question: {query}
 Understand the question properly.
 User the question briefly using the context & history provided
 Response in simple sentence and formaly. don't imagine any.
"""
  response = client.models.generate_content(
      model = model, contents = prompt)
  chat_history.append(f"User: {query}")
  chat_history.append(f"AI: {response.text}")
  return response.text

In [ ]:
display(Markdown(get_rag_response("what is anime industry? ")))

The anime industry manages production.

In [ ]:
while True:
  q = input("User: ")
  if q == "exit":
    break
  display(Markdown(get_rag_response(q)))

User: anime


Anime is Japanese animated content.

User: anime opening theme


An anime opening theme is the introductory song for Japanese animated content.

User: anime ending theme


...

User: anime recap


An anime recap summarizes previous events.

User: console gaming


Console gaming is playing video games on a dedicated gaming console.

User: exit
